# Project 1: An Analysis of Maternal Health Outcomes in America
## by Group 7: Mia Metni, Khadija Collymore and Christopher Wiggs
## 1. Research Question
### Are maternal mortality rates of women in America highest among black women (as opposed to women with other racial backgrounds)?

## 2. Hypothesis
### The elevated maternal mortality rates among black women in the United States compared to other racial groups stem from a complex interplay of factors:
- healthcare disparities, 
- socio-economic factors, 
- systemic biases, 
- limited access to quality healthcare, 
- economic inequality, 
- racial biases within healthcare systems 



In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
from scipy.stats import linregress

In [19]:
# Study data files
disease_indicators = "Resources/U.S._Chronic_Disease_Indicators__CDI_.csv"
maternal_death = "Resources/VSRR_Provisional_Maternal_Death_Counts_and_Rates_20240207.csv"
live_births_edu = "Resources/US_Live_Births_Race_Edu.csv"

In [20]:
# Read CSV files
disease_indicators_df = pd.read_csv(disease_indicators, dtype={'DataValueType': str}, low_memory=False) 
maternal_death_df = pd.read_csv(maternal_death, low_memory=False)
live_births_edu_df = pd.read_csv(live_births_edu, low_memory=False)

In [17]:
#Filter rows where race demographic is "black"
black_maternal_death = maternal_death_df[maternal_death_df['Subgroup'] == 'Black, Non-Hispanic']

In [18]:
black_maternal_death.to_csv('VSSR_Black_Maternal_Death_Counts.csv', index=False)

In [21]:
live_births_edu_df

,Notes,Mother's Single Race,Mother's Single Race Code,Mother's Education,Mother's Education Code,Births
0,NaN,American Indian or Alaska Native,1002-5,8th grade or less,1,1471.0
1,NaN,American Indian or Alaska Native,1002-5,9th through 12th grade with no diploma,2,6008.0
2,NaN,American Indian or Alaska Native,1002-5,High school graduate or GED completed,3,12969.0
3,NaN,American Indian or Alaska Native,1002-5,"Some college credit, but not a degree",4,8016.0
4,NaN,American Indian or Alaska Native,1002-5,"Associate degree (AA, AS)",5,2687.0
...,...,...,...,...,...,...
142,2003-2006 in CDC WONDER excluded Education and...,NaN,NaN,NaN,NaN,NaN
143,Standard Certificate of Live Birth. </p>,NaN,NaN,NaN,NaN,NaN
144,3. Single Race categories are not available fo...,NaN,NaN,NaN,NaN,NaN
145,"recoded to the Single Race category ""Not Avail...",NaN,NaN,NaN,NaN,NaN


In [22]:
# Drop specified columns
live_births_edu_df.drop(columns=["Mother's Single Race Code", "Notes", "Mother's Education Code"], inplace=True)

In [23]:
maternal_death_df

,Data As Of,Jurisdiction,Group,Subgroup,Year of Death,Month of Death,Time Period,Month Ending Date,Maternal Deaths,Live Births,Maternal Mortality Rate,Footnote
0,1/7/2024,United States,Total,Total,2019,1,12 month-ending,1/31/2019,660.0,3787776,17.4,NaN
1,1/7/2024,United States,Total,Total,2019,2,12 month-ending,2/28/2019,653.0,3783489,17.3,NaN
2,1/7/2024,United States,Total,Total,2019,3,12 month-ending,3/31/2019,657.0,3771682,17.4,NaN
3,1/7/2024,United States,Total,Total,2019,4,12 month-ending,4/30/2019,668.0,3772235,17.7,NaN
4,1/7/2024,United States,Total,Total,2019,5,12 month-ending,5/31/2019,706.0,3767999,18.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
565,1/7/2024,United States,Race and Hispanic origin,"White, Non-Hispanic",2023,5,12 month-ending,5/31/2023,309.0,1823992,16.9,NaN
566,1/7/2024,United States,Race and Hispanic origin,"White, Non-Hispanic",2023,6,12 month-ending,6/30/2023,296.0,1821388,16.3,NaN
567,1/7/2024,United States,Race and Hispanic origin,"White, Non-Hispanic",2023,7,12 month-ending,7/31/2023,287.0,1815013,15.8,NaN
568,1/7/2024,United States,Race and Hispanic origin,"White, Non-Hispanic",2023,8,12 month-ending,8/31/2023,286.0,1806212,15.8,NaN


In [24]:
# Rename column heading in maternal_death_df
maternal_death_df.rename(columns={"Subgroup": "Mother's Race"}, inplace=True)

# Rename column heading in live_births_edu_df
live_births_edu_df.rename(columns={"Mother's Single Race": "Mother's Race"}, inplace=True)

In [31]:
# Merge the two DataFrames
mh_merged_df = pd.merge(maternal_death_df, live_births_edu_df, on="Mother's Race", how="left")

In [32]:
mh_merged_df

,Data As Of,Jurisdiction,Group,Mother's Race,Year of Death,Month of Death,Time Period,Month Ending Date,Maternal Deaths,Live Births,Maternal Mortality Rate,Footnote,Mother's Education,Births
0,1/7/2024,United States,Total,Total,2019,1,12 month-ending,1/31/2019,660.0,3787776,17.4,NaN,NaN,NaN
1,1/7/2024,United States,Total,Total,2019,2,12 month-ending,2/28/2019,653.0,3783489,17.3,NaN,NaN,NaN
2,1/7/2024,United States,Total,Total,2019,3,12 month-ending,3/31/2019,657.0,3771682,17.4,NaN,NaN,NaN
3,1/7/2024,United States,Total,Total,2019,4,12 month-ending,4/30/2019,668.0,3772235,17.7,NaN,NaN,NaN
4,1/7/2024,United States,Total,Total,2019,5,12 month-ending,5/31/2019,706.0,3767999,18.7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,1/7/2024,United States,Race and Hispanic origin,"White, Non-Hispanic",2023,5,12 month-ending,5/31/2023,309.0,1823992,16.9,NaN,NaN,NaN
566,1/7/2024,United States,Race and Hispanic origin,"White, Non-Hispanic",2023,6,12 month-ending,6/30/2023,296.0,1821388,16.3,NaN,NaN,NaN
567,1/7/2024,United States,Race and Hispanic origin,"White, Non-Hispanic",2023,7,12 month-ending,7/31/2023,287.0,1815013,15.8,NaN,NaN,NaN
568,1/7/2024,United States,Race and Hispanic origin,"White, Non-Hispanic",2023,8,12 month-ending,8/31/2023,286.0,1806212,15.8,NaN,NaN,NaN
